In [1]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm



# lessons learned:
# 1. the problem should feasible, convex, for any given parameters
# 2. Make sure the there is no product of two parameters that to be inferred



# ----------------- 1. Define the MPC problem -----------------



from cvxpylayers.torch import CvxpyLayer
torch.set_default_dtype(torch.double)
torch.set_default_device('cpu')

# define the nominal dynamics
n = 4 # state dimension
m = 1 # control dimension
dt = 0.1 # time integrator
A_np = np.array([[1., dt],[0, 1.]]) # dynamics of the ego car
A = np.kron(np.eye(2), A_np) # dynamics of the two cars
B1 = np.array([0,dt,0,0]).reshape(-1,1) # control of the ego car
B2 = np.array([0,0,0,dt]).reshape(-1,1) # control of the other car

A_torch = torch.tensor(A)
B1_torch = torch.tensor(B1)
B2_torch = torch.tensor(B2)


# define the ground truth parameters of the cost function
# parameters:   
#               weight_on_velocity_cost, 
#               weight_on_acceleration_cost, 
#               desired_distance, 
#               desired_velocity,
# ground_truth_parameters = torch.tensor([1.0, 1.0, 0.5, 1.0])
# ground_truth_parameters_np = ground_truth_parameters.cpu().numpy()
control_limit = 100.0 # this is the constraint on the control
T = 5 # look-ahead horizon


# define the dynamics and cost function
def dynamics_torch(xt, ut, acceleration_of_other_car = torch.tensor([0.])):
    return A_torch @ xt + B1_torch @ ut + B2_torch @ acceleration_of_other_car

def dynamics_np(xt, ut, acceleration_of_other_car = np.array([0.])):
    return A @ xt + B1 @ ut + B2 @ acceleration_of_other_car

def cost(xt, ut):
    return (ground_truth_parameters*(xt.pow(2))).sum() + ut.pow(2).sum()


def construct_differentiable_mpc_problem(desired_speed):
    # belows are the parameters of the MPC problem
    x = cp.Parameter(n)
    inferred_parameters = cp.Parameter(3, nonneg=True) # this is the parameters of the problem
    acceleration_of_other_car = cp.Parameter(1)
    # belows are the decision variables of the MPC problem
    states = [cp.Variable(n) for _ in range(T)]
    controls = [cp.Variable(m) for _ in range(T)]
    # initial constraints
    constraints = [states[0] == x, 
        cp.norm(controls[0], 'inf') <= control_limit,
        states[0][0] <= states[0][2] - inferred_parameters[2]] 
    # initial objective
    # 
    objective = cp.square(inferred_parameters[2]-(states[0][2]-states[0][0])) +\
        cp.multiply(inferred_parameters[0], cp.square(states[0][1] - desired_speed))+\
        cp.multiply(inferred_parameters[1], cp.square(controls[0])) 
    for t in range(1, T):
        # objective
        #
        objective = cp.square(inferred_parameters[2]-(states[t][2]-states[t][0])) +\
            cp.multiply(inferred_parameters[0], cp.square(states[t][1] - desired_speed))+\
            cp.multiply(inferred_parameters[1], cp.square(controls[t])) 
        # dynamics constraints
        constraints += [states[t] == A @ states[t-1] +\
            B1 @ controls[t-1] +\
            B2 @ acceleration_of_other_car] 
        # control constraints
        constraints += [cp.norm(controls[t], 'inf') <= control_limit]
        constraints += [states[t][0] <= states[t][2]- inferred_parameters[2]] 
    problem = cp.Problem(cp.Minimize(objective), constraints)
    return CvxpyLayer(problem, variables=[controls[0]], parameters=[x, acceleration_of_other_car, inferred_parameters])

x0 = torch.tensor([0., 100, 1000, 100])
acceleration_of_other_car = torch.tensor([0.])
theta_0 = 1.0 # 
theta_1 = 10.0
safe_distance = 500.
desired_speed = 1340.

mpc_problem = construct_differentiable_mpc_problem(desired_speed)
acceleration = mpc_problem(x0, acceleration_of_other_car, torch.tensor([theta_0, theta_1, safe_distance]))
acceleration

(tensor([-1.1826]),)

In [9]:
from cvxpylayers.torch import CvxpyLayer
torch.set_default_dtype(torch.double)
torch.set_default_device('cpu')

# define the nominal dynamics
n = 4 # state dimension
m = 1 # control dimension
dt = 0.1 # time integrator
A_np = np.array([[1., dt],[0, 1.]]) # dynamics of the ego car
A = np.kron(np.eye(2), A_np) # dynamics of the two cars
B1 = np.array([0,dt,0,0]).reshape(-1,1) # control of the ego car
B2 = np.array([0,0,0,dt]).reshape(-1,1) # control of the other car

A_torch = torch.tensor(A)
B1_torch = torch.tensor(B1)
B2_torch = torch.tensor(B2)
def construct_mpc_problem(x, inferred_parameters, desired_speed, acceleration_of_other_car):
    # belows are the parameters of the MPC problem
    # x = cp.Parameter(n)
    # inferred_parameters = cp.Parameter(3, nonneg=True) # this is the parameters of the problem
    # acceleration_of_other_car = cp.Parameter(1)
    # belows are the decision variables of the MPC problem
    states = [cp.Variable(n) for _ in range(T)]
    controls = [cp.Variable(m) for _ in range(T)]
    # initial constraints
    constraints = [states[0] == x, 
        cp.norm(controls[0], 'inf') <= control_limit,
        states[0][0] <= states[0][2] - inferred_parameters[2]] 
    # initial objective
    # 
    objective = cp.square(inferred_parameters[2]-(states[0][2]-states[0][0])) +\
        cp.multiply(inferred_parameters[0], cp.square(states[0][1] - desired_speed))+\
        cp.multiply(inferred_parameters[1], cp.square(controls[0])) 
    for t in range(1, T):
        # objective
        #
        objective = cp.square(inferred_parameters[2]-(states[t][2]-states[t][0])) +\
            cp.multiply(inferred_parameters[0], cp.square(states[t][1] - desired_speed))+\
            cp.multiply(inferred_parameters[1], cp.square(controls[t])) 
        # dynamics constraints
        constraints += [states[t] == A @ states[t-1] +\
            B1 @ controls[t-1] +\
            B2 @ acceleration_of_other_car] 
        # control constraints
        constraints += [cp.norm(controls[t], 'inf') <= control_limit]
        constraints += [states[t][0] <= states[t][2]- inferred_parameters[2]] 
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve()
    print("sollution status: ", problem.status)
    return controls[0].value
    # return CvxpyLayer(problem, variables=[controls[0]], parameters=[x, acceleration_of_other_car, inferred_parameters])


In [10]:
control_limit = 100.0 # this is the constraint on the control
T = 5 # look-ahead horizon
x0 = torch.tensor([0., 1340, 1000, 1340])
x = x0
acceleration_of_other_car = np.array([0.])
theta_0 = 1.0 # 
theta_1 = 1.0 # control cost
safe_distance = 500.
desired_speed = 1340.
inferred_parameters = torch.tensor([theta_0, theta_1, safe_distance])
# acceleration_of_other_car = np.array([0.0])
acceleration = construct_mpc_problem(x, inferred_parameters, desired_speed,acceleration_of_other_car)
# acceleration = mpc_problem(x0, acceleration_of_other_car, torch.tensor([theta_0, theta_1, safe_distance]))
acceleration

sollution status:  optimal


array([100.])

In [11]:
acceleration

array([100.])